# 2. Feature Engineering

Partindo da base tratada e mergeada na etapa de Data Wrangling, nessa etapa será explorada a relação entre as variáveis para a criação de novas. 

Essa etapa e a análise exploratoria foram desenvolvidas iterativamente para que fossem encontradas boas variáveis e sua relação com as demais.

In [1]:
# Bibliotecas
import pandas as pd

****
### Ingestão dos dados


In [2]:
df = pd.read_csv('../data/2_intermediate/1_df_merged.csv')
df

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,...,DIAS_EMISSAO_VENCIMENTO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,DDD_1,TEMPO_VIDA_CLIENTE,DATA_CADASTRO
0,1661240395903230676,2018-09,16913.0,92.0,2018-09-16,2018-10-09,2018-10-08,22427.25,5.99,1,...,22,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1851,2013-08-22
1,1661240395903230676,2018-09,16913.0,92.0,2018-09-23,2018-10-15,2018-10-15,35608.11,5.99,0,...,22,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1858,2013-08-22
2,1661240395903230676,2018-10,236447.0,93.0,2018-10-08,2018-10-30,2018-10-29,17988.49,5.99,1,...,21,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1873,2013-08-22
3,1661240395903230676,2018-10,236447.0,93.0,2018-10-17,2018-11-07,2018-11-06,41998.20,6.99,1,...,20,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1882,2013-08-22
4,1661240395903230676,2018-10,236447.0,93.0,2018-10-21,2018-11-12,2018-11-12,35514.41,6.99,0,...,22,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1886,2013-08-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73472,3148228961195376491,2021-06,343507.0,134.0,2021-06-13,2021-07-26,2021-07-28,17302.05,4.99,-2,...,45,0,PJ,Comércio,BOL,MEDIO,36,0,2009,2015-12-13
73473,2439954154030259398,2021-06,392975.0,121.0,2021-06-14,2021-06-28,2021-06-29,19799.30,6.99,-1,...,15,0,PJ,Indústria,BOL,PEQUENO,13,0,713,2019-07-02
73474,2439954154030259398,2021-06,392975.0,121.0,2021-06-17,2021-07-01,2021-07-02,19913.15,11.99,-1,...,15,0,PJ,Indústria,BOL,PEQUENO,13,0,716,2019-07-02
73475,4799794458216343260,2021-06,70449.0,141.0,2021-06-14,2021-07-13,2021-07-09,2806.09,6.99,4,...,25,19,PJ,Indústria,OUTLOOK,MEDIO,20,1,532,2019-12-30


****
### Porcentagem de inadimplencias de um cliente dentro do seu próprio universo de emprestimos

Nas instruções especifica que o cliente é recorrente. Logo tenho um histórico dos emprestimos de cada cliente.

In [3]:
# Desejo acrescentar uma feature que indique a quantidade de emprestimos que o cliente foi inadimplente. A melhor forma é mostrar como porcentagem.
# Acrescentando a feature PORCENTAGEM_INADIMPLENCIA_HIST que indica qual a porcentagem de inadimplencias do cliente dentro do seu universo de emprestimos.

# Calcula o número de empréstimos inadimplentes para cada cliente
df_inadimplentes = df[df['INADIMPLENTE'] == 1].groupby('ID_CLIENTE').size().reset_index(name='NUM_INADIMPLENTES')

# Calcula o número total de empréstimos para cada cliente
df_total_emprestimos = df.groupby('ID_CLIENTE').size().reset_index(name='NUM_TOTAL_EMPRESTIMOS')

# Mescla os DataFrames de inadimplentes e total de empréstimos
df_porcent_inadimp = pd.merge(df_inadimplentes, df_total_emprestimos, on='ID_CLIENTE', how='outer')

# Calcula a porcentagem de empréstimos inadimplentes em relação ao total de empréstimos
df_porcent_inadimp['PORCENTAGEM_INADIMPLENCIA_HIST'] = (df_porcent_inadimp['NUM_INADIMPLENTES'] / df_porcent_inadimp['NUM_TOTAL_EMPRESTIMOS']) * 100

# Remove colunas desnecessárias
df_porcent_inadimp = df_porcent_inadimp.drop(['NUM_INADIMPLENTES', 'NUM_TOTAL_EMPRESTIMOS'], axis=1)

# Se não houver inadimplência, inputar 'zero'
df_porcent_inadimp['PORCENTAGEM_INADIMPLENCIA_HIST'].fillna(0, inplace=True)

df_porcent_inadimp

,ID_CLIENTE,PORCENTAGEM_INADIMPLENCIA_HIST
0,8784237149961904,0.423729
1,15070048310374146,25.000000
2,18719614953021044,33.333333
3,39547025441582855,1.515152
4,43266641219008038,100.000000
...,...,...
1178,9101617110967101591,0.000000
1179,9108733472015251498,0.000000
1180,9127318190908282126,0.000000
1181,9142266044170177362,0.000000


In [4]:
df = df.merge(df_porcent_inadimp, how='inner', on='ID_CLIENTE')
df

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,...,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,DDD_1,TEMPO_VIDA_CLIENTE,DATA_CADASTRO,PORCENTAGEM_INADIMPLENCIA_HIST
0,1661240395903230676,2018-09,16913.0,92.0,2018-09-16,2018-10-09,2018-10-08,22427.25,5.99,1,...,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1851,2013-08-22,8.411215
1,1661240395903230676,2018-09,16913.0,92.0,2018-09-23,2018-10-15,2018-10-15,35608.11,5.99,0,...,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1858,2013-08-22,8.411215
2,1661240395903230676,2018-10,236447.0,93.0,2018-10-08,2018-10-30,2018-10-29,17988.49,5.99,1,...,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1873,2013-08-22,8.411215
3,1661240395903230676,2018-10,236447.0,93.0,2018-10-17,2018-11-07,2018-11-06,41998.20,6.99,1,...,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1882,2013-08-22,8.411215
4,1661240395903230676,2018-10,236447.0,93.0,2018-10-21,2018-11-12,2018-11-12,35514.41,6.99,0,...,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1886,2013-08-22,8.411215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73472,3148228961195376491,2021-06,343507.0,134.0,2021-06-13,2021-07-26,2021-07-28,17302.05,4.99,-2,...,0,PJ,Comércio,BOL,MEDIO,36,0,2009,2015-12-13,0.000000
73473,2439954154030259398,2021-06,392975.0,121.0,2021-06-14,2021-06-28,2021-06-29,19799.30,6.99,-1,...,0,PJ,Indústria,BOL,PEQUENO,13,0,713,2019-07-02,0.000000
73474,2439954154030259398,2021-06,392975.0,121.0,2021-06-17,2021-07-01,2021-07-02,19913.15,11.99,-1,...,0,PJ,Indústria,BOL,PEQUENO,13,0,716,2019-07-02,0.000000
73475,4799794458216343260,2021-06,70449.0,141.0,2021-06-14,2021-07-13,2021-07-09,2806.09,6.99,4,...,19,PJ,Indústria,OUTLOOK,MEDIO,20,1,532,2019-12-30,0.000000


****
#### Outras features

- RENDA_ANTERIOR_POR_FUNCIONARIO
- VALOR_PAGAR_POR_RENDA_ANTERIOR
- VALOR_PAGAR_POR_NO_FUNCIONARIO

Agora desejo calcular a RENDA_MES_ANTERIOR por NO_FUNCIONARIOS

Mas dará erro porque é NO_FUNCIONARIOS = 0. Tal situação ocorre quando FLAG_PF = 'PF'.

Então, considerando que a PF é UM cliente, será atribuido NO_FUNCIONARIOS = 1 onde FLAG_PF = 'PF'

In [5]:
df.loc[df['FLAG_PF'] == 'PF', 'NO_FUNCIONARIOS'] = 1

In [6]:
df['RENDA_ANTERIOR_POR_FUNCIONARIO'] = df['RENDA_MES_ANTERIOR'] / df['NO_FUNCIONARIOS']
df['VALOR_PAGAR_POR_RENDA_ANTERIOR'] = df['VALOR_A_PAGAR'] / df['RENDA_MES_ANTERIOR']
df['VALOR_PAGAR_POR_NO_FUNCIONARIO'] = df['VALOR_A_PAGAR'] / df['NO_FUNCIONARIOS']

df

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,...,DOMINIO_EMAIL,PORTE,CEP_2_DIG,DDD_1,TEMPO_VIDA_CLIENTE,DATA_CADASTRO,PORCENTAGEM_INADIMPLENCIA_HIST,RENDA_ANTERIOR_POR_FUNCIONARIO,VALOR_PAGAR_POR_RENDA_ANTERIOR,VALOR_PAGAR_POR_NO_FUNCIONARIO
0,1661240395903230676,2018-09,16913.0,92.0,2018-09-16,2018-10-09,2018-10-08,22427.25,5.99,1,...,YAHOO,PEQUENO,65,9,1851,2013-08-22,8.411215,183.836957,1.326036,243.774457
1,1661240395903230676,2018-09,16913.0,92.0,2018-09-23,2018-10-15,2018-10-15,35608.11,5.99,0,...,YAHOO,PEQUENO,65,9,1858,2013-08-22,8.411215,183.836957,2.105369,387.044674
2,1661240395903230676,2018-10,236447.0,93.0,2018-10-08,2018-10-30,2018-10-29,17988.49,5.99,1,...,YAHOO,PEQUENO,65,9,1873,2013-08-22,8.411215,2542.440860,0.076078,193.424624
3,1661240395903230676,2018-10,236447.0,93.0,2018-10-17,2018-11-07,2018-11-06,41998.20,6.99,1,...,YAHOO,PEQUENO,65,9,1882,2013-08-22,8.411215,2542.440860,0.177622,451.593548
4,1661240395903230676,2018-10,236447.0,93.0,2018-10-21,2018-11-12,2018-11-12,35514.41,6.99,0,...,YAHOO,PEQUENO,65,9,1886,2013-08-22,8.411215,2542.440860,0.150200,381.875376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73472,3148228961195376491,2021-06,343507.0,134.0,2021-06-13,2021-07-26,2021-07-28,17302.05,4.99,-2,...,BOL,MEDIO,36,0,2009,2015-12-13,0.000000,2563.485075,0.050369,129.119776
73473,2439954154030259398,2021-06,392975.0,121.0,2021-06-14,2021-06-28,2021-06-29,19799.30,6.99,-1,...,BOL,PEQUENO,13,0,713,2019-07-02,0.000000,3247.727273,0.050383,163.630579
73474,2439954154030259398,2021-06,392975.0,121.0,2021-06-17,2021-07-01,2021-07-02,19913.15,11.99,-1,...,BOL,PEQUENO,13,0,716,2019-07-02,0.000000,3247.727273,0.050673,164.571488
73475,4799794458216343260,2021-06,70449.0,141.0,2021-06-14,2021-07-13,2021-07-09,2806.09,6.99,4,...,OUTLOOK,MEDIO,20,1,532,2019-12-30,0.000000,499.638298,0.039832,19.901348


****
### Novas variáveis a partir das variáveis de data

In [7]:
# Criando features com dia do mês, dia da semana, mês e ano de cada data

# Converter colunas com data de str para datetime
df['DATA_CADASTRO'] = pd.to_datetime(df['DATA_CADASTRO'])
df['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(df['DATA_EMISSAO_DOCUMENTO'])
df['DATA_PAGAMENTO'] = pd.to_datetime(df['DATA_PAGAMENTO'])
df['DATA_VENCIMENTO'] = pd.to_datetime(df['DATA_VENCIMENTO'])
df['SAFRA_REF'] = pd.to_datetime(df['SAFRA_REF'])

# Extrai dia do mês de cada data
df['DIA_MES_EMISSAO'] = df['DATA_EMISSAO_DOCUMENTO'].dt.day
df['DIA_MES_PAGAMENTO'] = df['DATA_PAGAMENTO'].dt.day
df['DIA_MES_VENCIMENTO'] = df['DATA_VENCIMENTO'].dt.day
df['DIA_MES_CADASTRO'] = df['DATA_CADASTRO'].dt.day
df['DIA_MES_SAFRA_REF'] = df['SAFRA_REF'].dt.day

# Extrai dia da semana de cada data
df['DIA_SEMANA_EMISSAO'] = df['DATA_EMISSAO_DOCUMENTO'].dt.day_name()
df['DIA_SEMANA_PAGAMENTO'] = df['DATA_PAGAMENTO'].dt.day_name()
df['DIA_SEMANA_VENCIMENTO'] = df['DATA_VENCIMENTO'].dt.day_name()
df['DIA_SEMANA_CADASTRO'] = df['DATA_CADASTRO'].dt.day_name()
df['DIA_SEMANA_SAFRA_REF'] = df['SAFRA_REF'].dt.day_name()

# Extrai mês de cada data
df['MES_EMISSAO_DOCUMENTO'] = df['DATA_EMISSAO_DOCUMENTO'].dt.month
df['MES_PAGAMENTO'] = df['DATA_PAGAMENTO'].dt.month
df['MES_VENCIMENTO'] = df['DATA_VENCIMENTO'].dt.month
df['MES_CADASTRO'] = df['DATA_CADASTRO'].dt.month
df['MES_SAFRA_REF'] = df['SAFRA_REF'].dt.month

# Extrai ano de cada data
df['ANO_EMISSAO_DOCUMENTO'] = df['DATA_EMISSAO_DOCUMENTO'].dt.year
df['ANO_PAGAMENTO'] = df['DATA_PAGAMENTO'].dt.year
df['ANO_VENCIMENTO'] = df['DATA_VENCIMENTO'].dt.year
df['ANO_CADASTRO'] = df['DATA_CADASTRO'].dt.year
df['ANO_SAFRA_REF'] = df['SAFRA_REF'].dt.year


df

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,...,MES_EMISSAO_DOCUMENTO,MES_PAGAMENTO,MES_VENCIMENTO,MES_CADASTRO,MES_SAFRA_REF,ANO_EMISSAO_DOCUMENTO,ANO_PAGAMENTO,ANO_VENCIMENTO,ANO_CADASTRO,ANO_SAFRA_REF
0,1661240395903230676,2018-09-01,16913.0,92.0,2018-09-16,2018-10-09,2018-10-08,22427.25,5.99,1,...,9,10,10,8,9,2018,2018,2018,2013,2018
1,1661240395903230676,2018-09-01,16913.0,92.0,2018-09-23,2018-10-15,2018-10-15,35608.11,5.99,0,...,9,10,10,8,9,2018,2018,2018,2013,2018
2,1661240395903230676,2018-10-01,236447.0,93.0,2018-10-08,2018-10-30,2018-10-29,17988.49,5.99,1,...,10,10,10,8,10,2018,2018,2018,2013,2018
3,1661240395903230676,2018-10-01,236447.0,93.0,2018-10-17,2018-11-07,2018-11-06,41998.20,6.99,1,...,10,11,11,8,10,2018,2018,2018,2013,2018
4,1661240395903230676,2018-10-01,236447.0,93.0,2018-10-21,2018-11-12,2018-11-12,35514.41,6.99,0,...,10,11,11,8,10,2018,2018,2018,2013,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73472,3148228961195376491,2021-06-01,343507.0,134.0,2021-06-13,2021-07-26,2021-07-28,17302.05,4.99,-2,...,6,7,7,12,6,2021,2021,2021,2015,2021
73473,2439954154030259398,2021-06-01,392975.0,121.0,2021-06-14,2021-06-28,2021-06-29,19799.30,6.99,-1,...,6,6,6,7,6,2021,2021,2021,2019,2021
73474,2439954154030259398,2021-06-01,392975.0,121.0,2021-06-17,2021-07-01,2021-07-02,19913.15,11.99,-1,...,6,7,7,7,6,2021,2021,2021,2019,2021
73475,4799794458216343260,2021-06-01,70449.0,141.0,2021-06-14,2021-07-13,2021-07-09,2806.09,6.99,4,...,6,7,7,12,6,2021,2021,2021,2019,2021


In [8]:
df.columns

Index(['ID_CLIENTE', 'SAFRA_REF', 'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS',
       'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO', 'DATA_VENCIMENTO',
       'VALOR_A_PAGAR', 'TAXA', 'DIAS_ATRASO', 'INADIMPLENTE',
       'DIAS_EMISSAO_PAGAMENTO', 'DIAS_EMISSAO_VENCIMENTO', 'DDD', 'FLAG_PF',
       'SEGMENTO_INDUSTRIAL', 'DOMINIO_EMAIL', 'PORTE', 'CEP_2_DIG', 'DDD_1',
       'TEMPO_VIDA_CLIENTE', 'DATA_CADASTRO', 'PORCENTAGEM_INADIMPLENCIA_HIST',
       'RENDA_ANTERIOR_POR_FUNCIONARIO', 'VALOR_PAGAR_POR_RENDA_ANTERIOR',
       'VALOR_PAGAR_POR_NO_FUNCIONARIO', 'DIA_MES_EMISSAO',
       'DIA_MES_PAGAMENTO', 'DIA_MES_VENCIMENTO', 'DIA_MES_CADASTRO',
       'DIA_MES_SAFRA_REF', 'DIA_SEMANA_EMISSAO', 'DIA_SEMANA_PAGAMENTO',
       'DIA_SEMANA_VENCIMENTO', 'DIA_SEMANA_CADASTRO', 'DIA_SEMANA_SAFRA_REF',
       'MES_EMISSAO_DOCUMENTO', 'MES_PAGAMENTO', 'MES_VENCIMENTO',
       'MES_CADASTRO', 'MES_SAFRA_REF', 'ANO_EMISSAO_DOCUMENTO',
       'ANO_PAGAMENTO', 'ANO_VENCIMENTO', 'ANO_CADASTRO', 'ANO_SAF

****
### Persistência dos dados

#### i. Dados para a Análise Exploratória

In [9]:
# Salva dataframe final para a etapa de Análise Exploratória
df.to_csv('../data/2_intermediate/2_df_feat_eng.csv', index=False)

#### ii. Features para inferência

Armazenamento de features relacionadas à perfil do cliente, como cadastro e informações gerais.

Poderiam ser armazenadas em banco para inferência futura.

In [10]:
df_features = df[['ID_CLIENTE', 'PORCENTAGEM_INADIMPLENCIA_HIST', 'DDD', 'FLAG_PF', 'SEGMENTO_INDUSTRIAL', 'DOMINIO_EMAIL', 'PORTE', 'CEP_2_DIG', 'DDD_1', 'DATA_CADASTRO']]
df_features = df_features.drop_duplicates().reset_index(drop=True)

df_features

,ID_CLIENTE,PORCENTAGEM_INADIMPLENCIA_HIST,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,DDD_1,DATA_CADASTRO
0,1661240395903230676,8.411215,99,PJ,Serviços,YAHOO,PEQUENO,65,9,2013-08-22
1,8274986328479596038,28.571429,31,PJ,Comércio,YAHOO,MEDIO,77,3,2017-01-25
2,345447888460137901,9.523810,75,PJ,Serviços,HOTMAIL,PEQUENO,48,7,2000-08-15
3,1003144834589372198,0.000000,49,PJ,Serviços,OUTLOOK,PEQUENO,89,4,2017-08-06
4,324916756972236008,3.703704,88,PJ,Serviços,GMAIL,GRANDE,62,8,2011-02-14
...,...,...,...,...,...,...,...,...,...,...
1178,8736788139020105764,100.000000,0,PJ,Serviços,HOTMAIL,MEDIO,13,0,2021-02-04
1179,3148228961195376491,0.000000,0,PJ,Comércio,BOL,MEDIO,36,0,2015-12-13
1180,2439954154030259398,0.000000,0,PJ,Indústria,BOL,PEQUENO,13,0,2019-07-02
1181,4799794458216343260,0.000000,19,PJ,Indústria,OUTLOOK,MEDIO,20,1,2019-12-30


In [11]:
df_features.to_csv('../data/3_primary/df_features_info.csv', index=False)

****
## Resumo

- Novas features:
    - DIAS_ATRASO: 
        - apenas para análise
    - DIAS_EMISSAO_PAGAMENTO: 
        - apenas para análise
    - DIAS_EMISSAO_VENCIMENTO
    - TEMPO_VIDA_CLIENTE: 
        - Tempo de vida do cliente até a emissão do respectivo empréstimo
    - PORCENTAGEM_INADIMPLENCIA_HIST
    - RENDA_ANTERIOR_POR_FUNCIONARIO
    - VALOR_PAGAR_POR_RENDA_ANTERIOR
    - VALOR_PAGAR_POR_NO_FUNCIONARIO
    - DIA_MES_EMISSAO
    - DIA_MES_PAGAMENTO: 
        - apenas para análise
    - DIA_MES_VENCIMENTO
    - DIA_MES_CADASTRO
    - DIA_MES_SAFRA_REF
    - DIA_SEMANA_EMISSAO
    - DIA_SEMANA_PAGAMENTO: 
        - apenas para análise
    - DIA_SEMANA_VENCIMENTO
    - DIA_SEMANA_CADASTRO
    - DIA_SEMANA_SAFRA_REF
    - MES_EMISSAO_DOCUMENTO
    - MES_PAGAMENTO: 
        - apenas para análise
    - MES_VENCIMENTO
    - MES_CADASTRO
    - MES_SAFRA_REF
    - ANO_EMISSAO_DOCUMENTO
    - ANO_PAGAMENTO: 
        - apenas para análise
    - ANO_VENCIMENTO
    - ANO_CADASTRO
    - ANO_SAFRA_REF
    